Taking the LAIGLE spectroscopic catalog to make a full catalog with selected LCBGs

| Date | Person | Change |
| :- | :- | :--------: |
04/24/2019  |  L. Hunt  |  <ul><li>Initial Version</li><li>Import CSV</li><li>Use kcorrect to correct get absolut magnitudes</li></ul>
04/26/2019 | L. Hunt | <ul><li>Runs faster on setesh</li><li>Once get kcorrections will be faster to work with</li><li>KCORRECT should work with correct filter list (Added new filters to kcorrect dir)</li></ul>
05/02/2019 | L. Hunt | updated to run with spec catalog matched to COSMOS 08 ID

Import number and plotting modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Import astronomy modules

In [2]:
import astropy as ap
from astropy import units as u
import kcorrect
import kcorrect.utils as ut
from astropy.cosmology import FlatLambdaCDM

Import basic modules

In [3]:
from itertools import combinations
import os
import datetime

In [4]:
def make_kcorr_filt_template(dataframe):
    '''This task will make a kcorrect filter template from a dataframe that optimizes the number of objects with detections in a subset of filters. In this case the dataframe should contain cfht, subaru, and irac wideband filters. '''
    kcordir=os.environ["KCORRECT_DIR"]
    lambdar_to_kcorr={'u_MAG_AUTO':'capak_cfht_megaprime_sagem_u.par',
                      'B_MAG_AUTO':'capak_subaru_suprimecam_B.par',
                      'V_MAG_AUTO':'capak_subaru_suprimecam_V.par',
                      'r_MAG_AUTO':'capak_subaru_suprimecam_r.par',
                      'ip_MAG_AUTO':'capak_subaru_suprimecam_i.par',
                      'zpp_MAG_AUTO':'subaru_suprimecam_zpp.par',
                      'Y_MAG_AUTO':'vircam_Y.par',
                      'J_MAG_AUTO':'vircam_J.par',
                      'H_MAG_AUTO':'vircam_H.par',
                      'Ks_MAG_AUTO':'vircam_K.par'}
    numb1=0
    numb2=0
    numb3=0
    numb4=0
    numb5=0
    flist1=[]
    flist2=[]
    flist3=[]
    flist4=[]
    flist5=[]
    ilist1=[]
    ilist2=[]
    ilist3=[]
    ilist4=[]
    ilist5=[]
    kcor_template=kcordir+'/data/templates/temp_filt_list.dat'
    for x in combinations(list(dataframe),5):
        if len(dataframe[(dataframe[x[0]]<40)&
                         (dataframe[x[1]]<40)&
                         (dataframe[x[2]]<40)&
                         (dataframe[x[3]]<40)&
                         (dataframe[x[4]]<40)]) > numb1:
            ilist5=ilist4
            ilist4=ilist3
            ilist3=ilist2
            ilist2=ilist1
            ilist1=dataframe[(dataframe[x[0]]<40)&
                             (dataframe[x[1]]<40)&
                             (dataframe[x[2]]<40)&
                             (dataframe[x[3]]<40)&
                             (dataframe[x[4]]<40)].index.tolist()
            numb5=numb4
            numb4=numb3
            numb3=numb2
            numb2=numb1
            numb1=len(ilist1)
            flist5=flist4
            flist4=flist3
            flist3=flist2
            flist2=flist1
            flist1=x
    with open(kcor_template,'w') as file:
        file.write('KCORRECT_DIR\n')
        for filt in flist1:
            file.write('data/filters/cosmos_filters/'+lambdar_to_kcorr[filt]+'\n')
    return flist1,kcor_template

In [8]:
cosmo=FlatLambdaCDM(H0=70,Om0=0.3)

In [7]:
kcordir=os.environ["KCORRECT_DIR"]
catbasedir=os.environ["COSMOS_DIR"].split('Original')[0]+'/Final_Catalogs'

In [11]:
print(catbasedir)

/Users/lucashunt/ASTRODATA/LCBG_LUMINOSITY_FUNCTION/COSMOS_CATALOGS/Final_Catalogs


In [9]:
Spec_Cat=pd.read_csv(catbasedir+'/final_spec_catalog.csv')

In [10]:
Spec_Cat['final_z']

0         0.038600
1         0.369500
2         0.160000
3         2.334300
4         1.391600
5         1.779200
6         2.199100
7         2.440700
8         0.207900
9         1.206000
10        0.592700
11        1.956000
12        1.104400
13        2.763400
14        0.880300
15        1.953900
16        2.914500
17        2.480900
18        2.207700
19        1.636300
20        0.297600
21        3.267700
22        3.770400
23        0.306500
24        1.588600
25        0.868500
26        4.951600
27        2.041700
28        1.280900
29        1.274200
            ...   
536050    0.375600
536051    0.379900
536052    0.809600
536053    0.815000
536054    0.563800
536055    0.247446
536056    1.122900
536057    1.223600
536058    1.207700
536059    0.566200
536060    1.315600
536061    1.148000
536062    0.926736
536063    0.697800
536064    0.430000
536065    0.681200
536066    0.498200
536067    0.344023
536068    0.705210
536069    0.197800
536070    0.367300
536071    0.

In [8]:
magnitude_columns=[filt for filt in Spec_Cat.columns.values if '_mag_' in filt.lower()]
magnitude_error_columns=[filt for filt in Spec_Cat.columns.values if 'err' in filt.lower()]

In [9]:
kcorr_filt_list,kcor_template_filter=make_kcorr_filt_template(Spec_Cat[magnitude_columns])

In [10]:
print(kcor_template_filter)

/home/lrhunt/programs/kcorrect/data/templates/temp_filt_list.dat


In [11]:
maggies_filt_list=[filt.split('_')[0]+
                   '_maggies' for filt in kcorr_filt_list]

invervar_filt_list=[filt.split('_')[0]+
                    '_invervar' for filt in kcorr_filt_list]

synthetic_maggies_filt_list=[filt.split('_')[0]+
                             '_synthetic_maggies' for filt in 
                             kcorr_filt_list]

rf_synthetic_maggies_filt_list=[filt.split('_')[0]+
                             '0_synthetic_maggies' for filt in 
                             kcorr_filt_list]

In [12]:
numbers=np.arange(1,7,1)
for string in ['c'+str(number) for number in numbers]:
    Spec_Cat[string]=np.nan

In [13]:
print('Converting to Maggies')
for column in magnitude_columns:
    Spec_Cat[column.split('_')[0]
             +'_maggies']=ut.mag2maggies(Spec_Cat[column])
    Spec_Cat[column.split('_')[0]
             +'_invervar'
            ]=ut.invariance(Spec_Cat[column.split('_')[0]
                                     +'_maggies'],
                            Spec_Cat[column.split('_')[0]
                                     +'_MAGERR_AUTO']
                           )

Converting to Maggies


In [14]:
for i in range(0,len(synthetic_maggies_filt_list)):
    Spec_Cat[synthetic_maggies_filt_list[i]]=np.nan
    Spec_Cat[rf_synthetic_maggies_filt_list[i]]=np.nan

In [15]:
kcorrect.load_templates()
kcorrect.load_filters(kcor_template_filter)

In [16]:
indexes=Spec_Cat.index.values

In [17]:
time=datetime.datetime.now()
for i in indexes:
    Spec_Cat.loc[i,'c1':'c6']=kcorrect.fit_nonneg(np.array(Spec_Cat.loc[i,'final_z'],
                                                           dtype=float),
                                                  np.array(Spec_Cat.loc[i,maggies_filt_list],
                                                           dtype=float),
                                                  np.array(Spec_Cat.loc[i,invervar_filt_list],
                                                           dtype=float))
    Spec_Cat.loc[i,synthetic_maggies_filt_list]=kcorrect.reconstruct_maggies(Spec_Cat.loc[i,'c1':'c6'])[1:]
    Spec_Cat.loc[i,rf_synthetic_maggies_filt_list]=kcorrect.reconstruct_maggies(Spec_Cat.loc[i,'c1':'c6'],redshift=0)[1:]
    if i%10000==0:
        print('k-correction for {} sources done'.format(i))
        print(datetime.datetime.now()-time)
        time=datetime.datetime.now()

k-correction for 0 sources done
0:00:00.487218
k-correction for 10000 sources done
0:04:21.542996
k-correction for 20000 sources done
0:04:09.200802
k-correction for 30000 sources done
0:04:00.204736
k-correction for 40000 sources done
0:04:00.677966
k-correction for 50000 sources done
0:04:02.153853
k-correction for 60000 sources done
0:04:13.744376
k-correction for 70000 sources done
0:04:39.460127
k-correction for 80000 sources done
0:04:44.495168
k-correction for 90000 sources done
0:04:44.756752
k-correction for 100000 sources done
0:04:47.949166
k-correction for 110000 sources done
0:04:47.845044
k-correction for 120000 sources done
0:04:42.174257
k-correction for 130000 sources done
0:04:45.460870
k-correction for 140000 sources done
0:04:43.786725
k-correction for 150000 sources done
0:04:44.691609
k-correction for 160000 sources done
0:04:49.044942
k-correction for 170000 sources done
0:04:47.670888
k-correction for 180000 sources done
0:04:45.051776
k-correction for 190000 so

In [18]:
Spec_Cat['UJ0_synthetic_maggies']=np.nan
Spec_Cat['BJ0_synthetic_maggies']=np.nan
Spec_Cat['VJ0_synthetic_maggies']=np.nan

In [19]:
kcorrect.load_templates()
kcorrect.load_filters(kcordir+'/data/templates/bessell_ubv.dat')

In [20]:
for i in indexes:
    Spec_Cat.loc[i,
                 ['UJ0_synthetic_maggies',
                 'BJ0_synthetic_maggies',
                  'VJ0_synthetic_maggies']
                ]=kcorrect.reconstruct_maggies(Spec_Cat.loc[i,
                                                            'c1':'c6']
                                                                     ,redshift=0)[1:]
    if i%10000==0:
        print('k-correction for {} sources done'.format(i))
        print(datetime.datetime.now()-time)
        time=datetime.datetime.now()

k-correction for 0 sources done
0:02:32.352353
k-correction for 10000 sources done
0:00:44.129050
k-correction for 20000 sources done
0:00:44.574936
k-correction for 30000 sources done
0:00:48.057698
k-correction for 40000 sources done
0:00:47.921068
k-correction for 50000 sources done
0:00:47.585492
k-correction for 60000 sources done
0:00:46.030134
k-correction for 70000 sources done
0:00:46.942641
k-correction for 80000 sources done
0:00:46.171763
k-correction for 90000 sources done
0:00:48.628078
k-correction for 100000 sources done
0:00:48.403447
k-correction for 110000 sources done
0:00:45.697005
k-correction for 120000 sources done
0:00:46.535397
k-correction for 130000 sources done
0:00:46.138684
k-correction for 140000 sources done
0:00:48.697283
k-correction for 150000 sources done
0:00:47.007969
k-correction for 160000 sources done
0:00:47.587646
k-correction for 170000 sources done
0:00:43.780043
k-correction for 180000 sources done
0:00:44.246525
k-correction for 190000 so

In [21]:
Spec_Cat[['UJ0_synthetic_AB_mag',
          'BJ0_synthetic_AB_mag',
          'VJ0_synthetic_AB_mag']]=-2.5*np.log10(
    Spec_Cat[['UJ0_synthetic_maggies',
              'BJ0_synthetic_maggies',
              'VJ0_synthetic_maggies']])
Spec_Cat['UJ0_synthetic_vega_mag']=Spec_Cat['UJ0_synthetic_AB_mag']-0.79
Spec_Cat['BJ0_synthetic_vega_mag']=Spec_Cat['BJ0_synthetic_AB_mag']+0.09
Spec_Cat['VJ0_synthetic_vega_mag']=Spec_Cat['VJ0_synthetic_AB_mag']-0.02

In [33]:
[f.split('_')[0]+'_kcorr_BJ0' 
          for f in 
          synthetic_maggies_filt_list]

['B_kcorr_BJ0', 'V_kcorr_BJ0', 'r_kcorr_BJ0', 'ip_kcorr_BJ0', 'zpp_kcorr_BJ0']

In [30]:
Spec_Cat[[f.split('_')[0]+'_kcorr_BJ0' 
          for f in 
          synthetic_maggies_filt_list]]=-2.5*np.log10(
    Spec_Cat[synthetic_maggies_filt_list]/
    np.stack((Spec_Cat['BJ0_synthetic_maggies'],
              Spec_Cat['BJ0_synthetic_maggies'],
              Spec_Cat['BJ0_synthetic_maggies'],
              Spec_Cat['BJ0_synthetic_maggies'],
              Spec_Cat['BJ0_synthetic_maggies']),axies=-1))

KeyError: 'BJ0_synthetic_maggies'

In [ ]:
Spec_Cat.to_csv(catbasedir+'/Laigle_Cat_With_kcorrections.csv',index=False)